In [17]:
import pandas as pd
import numpy as np
import nltk
#nltk.download()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

In [18]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to C:\Users\OGUNJOBI
[nltk_data]     Victor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\OGUNJOBI
[nltk_data]     Victor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\OGUNJOBI
[nltk_data]     Victor\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\OGUNJOBI Victor\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [132]:
df = pd.read_csv("./clean_data.csv")

In [133]:
df.head()

,Unnamed: 0,Job title,Salary,Date,Company Location,Company Name,city,state,label
0,0,HYBRID - Electronic and Digital Design Special...,$27.03 an hour,19 days ago,"Remote in Columbus, OH 43215",State of Ohio Jobs,Remote in Columbus,OH 43215,negative
1,1,Layout Design Artist,$20.76 an hour,1 day ago,"Hybrid remote in Columbus, OH 43211",State of Ohio Jobs,Hybrid remote in Columbus,OH 43211,negative
2,2,Design Researcher,NaN,8 days ago,NaN,Central Mutual Insurance Company,NaN,NaN,negative
3,12,Design Consultant,"Up to $180,000 a year",30+ days ago,"Dublin, OH 43016",Bassett Furniture,Dublin,OH 43016,negative
4,13,"Product Design, Associate",NaN,4 days ago,NaN,"JPMorgan Chase Bank, N.A.",NaN,NaN,negative


# User input

In [198]:
user_input = str(input('Enter any job you want recommendation(s) on: '))
from_user = pd.DataFrame(data=[user_input], columns = ["Text"])
from_user.index=range(len(from_user.index))
from_user

Enter any job you want recommendation(s) on: Data Scientist


,Text
0,Data Scientist


In [199]:
#Replace Nan values
df.fillna("", inplace = True)

In [200]:
#Creating the Jobs corpus

df['Text'] = df['Job title'].map(str) + ' ' + df['Salary'] + ' ' + df['Date'] + ' ' + df['Company Location'] + ' ' + df['Company Name'] + ' ' + df['city']+ ' ' + df['state']
df.head(10)

,Unnamed: 0,Job title,Salary,Date,Company Location,Company Name,city,state,label,Text
0,0,HYBRID - Electronic and Digital Design Special...,$27.03 an hour,19 days ago,"Remote in Columbus, OH 43215",State of Ohio Jobs,Remote in Columbus,OH 43215,negative,HYBRID - Electronic and Digital Design Special...
1,1,Layout Design Artist,$20.76 an hour,1 day ago,"Hybrid remote in Columbus, OH 43211",State of Ohio Jobs,Hybrid remote in Columbus,OH 43211,negative,Layout Design Artist $20.76 an hour 1 day ago ...
2,2,Design Researcher,,8 days ago,,Central Mutual Insurance Company,,,negative,Design Researcher 8 days ago Central Mutual ...
3,12,Design Consultant,"Up to $180,000 a year",30+ days ago,"Dublin, OH 43016",Bassett Furniture,Dublin,OH 43016,negative,"Design Consultant Up to $180,000 a year 30+ da..."
4,13,"Product Design, Associate",,4 days ago,,"JPMorgan Chase Bank, N.A.",,,negative,"Product Design, Associate 4 days ago JPMorga..."
5,14,"Artist, Graphic Retail",,1 day ago,,"Giant Eagle, Inc.",,,negative,"Artist, Graphic Retail 1 day ago Giant Eagle..."
6,15,Product Design Engineer,,30+ days ago,"Remote in London, OH",Harness.io,Remote in London,OH,negative,Product Design Engineer 30+ days ago Remote i...
7,16,Designer,,30+ days ago,,McGraw Hill,,,negative,Designer 30+ days ago McGraw Hill
8,17,Pricing Design Architect III - REMOTE/HYBRID,,15 days ago,"Remote in Columbus, OH",FedEx Supply Chain,Remote in Columbus,OH,negative,Pricing Design Architect III - REMOTE/HYBRID ...
9,18,Design Consultant,,30+ days ago,,The Cleary Company,,,negative,Design Consultant 30+ days ago The Cleary Co...


In [201]:
data = df[['Unnamed: 0', 'Text', 'Job title']]
data = data.fillna(' ')
data.head()

,Unnamed: 0,Text,Job title
0,0,HYBRID - Electronic and Digital Design Special...,HYBRID - Electronic and Digital Design Special...
1,1,Layout Design Artist $20.76 an hour 1 day ago ...,Layout Design Artist
2,2,Design Researcher 8 days ago Central Mutual ...,Design Researcher
3,12,"Design Consultant Up to $180,000 a year 30+ da...",Design Consultant
4,13,"Product Design, Associate 4 days ago JPMorga...","Product Design, Associate"


In [202]:
data.rename(columns={'Unnamed: 0':"Job.ID"}, inplace = True)

In [203]:
data

,Job.ID,Text,Job title
0,0,HYBRID - Electronic and Digital Design Special...,HYBRID - Electronic and Digital Design Special...
1,1,Layout Design Artist $20.76 an hour 1 day ago ...,Layout Design Artist
2,2,Design Researcher 8 days ago Central Mutual ...,Design Researcher
3,12,"Design Consultant Up to $180,000 a year 30+ da...",Design Consultant
4,13,"Product Design, Associate 4 days ago JPMorga...","Product Design, Associate"
...,...,...,...
2393,5812,"Extension Educator, 4-H Youth Development 1 d...","Extension Educator, 4-H Youth Development"
2394,5813,Facilities Management & Services Worker II (Pr...,Facilities Management & Services Worker II (Pr...
2395,5814,"Director, Cooperative Extension 30+ days ago ...","Director, Cooperative Extension"
2396,5815,"Associate Director, Programs, Family and Consu...","Associate Director, Programs, Family and Consu..."


In [204]:
stopword = stopwords.words('english')
stopword_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

In [205]:
# Create word tokens
def token_txt(token):
    return token not in stopword_ and token not in list(string.punctuation) and len(token) > 2   
  
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "", text)
  text = re.sub("(\\d|\\W)+", " ", text) 
  text = text.replace("nbsp", "")
  clean_text = [wn.lemmatize(word, pos = "v") for word in word_tokenize(text.lower()) if token_txt(word)]
  clean_text2 = [word for word in clean_text if token_txt(word)]
  return " ".join(clean_text2)

In [206]:
data['Text'] = data['Text'].apply(clean_txt) 

# Term Frequency - Inverse Document Frequency

In [207]:
tfidf_vect = TfidfVectorizer()

# Fitting and transforming the vector
tfidf_comb = tfidf_vect.fit_transform((data['Text'])) 
tfidf_comb

<2398x2742 sparse matrix of type '<class 'numpy.float64'>'
	with 21926 stored elements in Compressed Sparse Row format>

# Building the Recommender Systems

In [208]:
#Computing the Cosine Similarity using TF-IDF

user_tfidf = tfidf_vect.transform(from_user['Text'])
cos_sim_tfidf = map(lambda x: cosine_similarity(user_tfidf, x), tfidf_comb)

In [209]:
cos_sim_tfidf

In [210]:
rec1 = list(cos_sim_tfidf)

In [211]:
def get_recommendation(top, the_data, scores):
  recommendation = pd.DataFrame(columns = ['Job_ID',  'Job title', 'Score'], dtype=object)
  count = 0
  for i in top:
      recommendation.at[count, 'Job_ID'] = the_data['Job.ID'][i]
      recommendation.at[count, 'Job title'] = the_data['Job title'][i]
      recommendation.at[count, 'Score'] =  scores[count]
      count += 1
  return recommendation

In [212]:
#Using TF-IDF for recommendation

top10_tfidf = sorted(range(len(rec1)), key = lambda i: rec1[i], reverse = True)[:10]
top10_tfidf

[1232, 2161, 2160, 389, 404, 358, 1084, 1090, 1951, 1946]

In [213]:
list_scores_tfidf = [rec1[i][0][0] for i in top10_tfidf]
list_scores_tfidf

[0.5243544326746316,
 0.4824940092858012,
 0.4688946858212054,
 0.42315926868187914,
 0.41095137578164626,
 0.40511696458428315,
 0.3813946657141657,
 0.3445208903420802,
 0.33862190210890625,
 0.3378096293133905]

In [214]:
get_recommendation(top10_tfidf, data, list_scores_tfidf)

,Job_ID,Job title,Score
0,4324,Data Scientist,0.524354
1,5564,HR Analytics – Data Scientist (Remote),0.482494
2,5563,Data Scientist,0.468895
3,2190,Data Scientist,0.423159
4,2221,Research Scientist,0.410951
5,2151,Research Scientist,0.405117
6,4094,Data Analytics Scientist,0.381395
7,4100,"Consultant, Analytic Scientist",0.344521
8,5229,Research Scientist,0.338622
9,5224,Research Scientist,0.33781


In [215]:
#Using Count Vectorizer for recommendation

count_vect = CountVectorizer()

# Fitting and transforming the vectorizer
count_comb = count_vect.fit_transform((data['Text'])) #fitting and transforming the vector
count_comb

<2398x2742 sparse matrix of type '<class 'numpy.int64'>'
	with 21926 stored elements in Compressed Sparse Row format>

In [216]:
user_count_countvec = count_vect.transform(from_user['Text'])
cos_sim_count_countvec = map(lambda x: cosine_similarity(user_count_countvec, x), count_comb)
count_vec1 = list(cos_sim_count_countvec)

In [217]:
top10_count_vec_count = sorted(range(len(count_vec1)), key = lambda i: count_vec1[i], reverse = True)[:10]
list_scores_vec_count = [count_vec1[i][0][0] for i in top10_count_vec_count]
get_recommendation(top10_count_vec_count, data, list_scores_vec_count)

,Job_ID,Job title,Score
0,5563,Data Scientist,0.5
1,4094,Data Analytics Scientist,0.392232
2,4324,Data Scientist,0.377964
3,2190,Data Scientist,0.365148
4,4658,Data Owner -Regulatory External Data Services-...,0.365148
5,4908,Sr Data Architect - Data Lake & Analytics - Na...,0.365148
6,5564,HR Analytics – Data Scientist (Remote),0.353553
7,112,"Principal Data Architect, Data Lake & Analytics",0.333333
8,103,Data Architect - Data Warehouse & MPP - Nation...,0.324443
9,104,Sr Data Architect - Data Warehouse & MPP - Nat...,0.324443
